# Conversion of CMU movies to IMDB IDs and TMDB IDs
We would like to convert the references from the CMU movies dataset to Imdb ID (`tconst` values), so that we can link the ratings value and the number of ratings to each movies. Aditionally, we want to have the TMDB IDs linked to fetch the order of the character's importance role in a movie. We try two approaches:
1. **Use Wikipedia IDs:** each movie entry has a unique Wikipedia ID to access a Wikidata value, and then we use the Wikidata value to access the Imdb ID. We used the Wikimedia API to access these values.
2. **Use Freebase IDs:** use the unique freebase value the CMU dataset and wikipedia queries to access the Imdb ID. (For ease of computation, we'll only consider TMDB values in this case)

Both conversion methods may yield varying results, because some pages might have been modified overtime since 2012, references deleted. We maximize our conversion rates by trying both method. We then compare the comversion success rate of the two approaches.

In [1]:
import requests
import pandas as pd
import numpy as np
from IPython.display import clear_output

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Import token from config.py
from config import WIKI_API_TOKEN

In [3]:
raw_dir = './raw_data/'
tmp_dir = './tmp_data/'
processed_dir = './processed_data/'

# Import the movie data
movies_dir = raw_dir + 'CMU/movie.metadata.tsv'

# Read the file into a DataFrame, add headers
movie_df = pd.read_csv(movies_dir, sep='\t', header=None)

# Add column names deduced from README
movie_df.columns = ['wikipedia_ID', 'freebase_ID', 'name', 'release_date', 'box_office', 'runtime', 'languages', 'countries', 'genres']

# Set the index to wiki_ID
movie_df.set_index('wikipedia_ID', inplace=True)
display(movie_df)

,freebase_ID,name,release_date,box_office,runtime,languages,countries,genres
wikipedia_ID,,,,,,,,
975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...
35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"


## Method 1: From Wikipedia page ID to IMDB
### Part 1: Wikipedia ID to Wikidata ID
Fecth the wikidata ID of the wikipedia page ID using the wikipedia API, for all movies in the CMU database

#### Test for single ID
We are requesting for a single movie wikipedia ID to see if authentication is working.

In [4]:
# Request setup
base_url = 'https://en.wikipedia.org/w/api.php'

headers = {
    "Authorization": "Bearer {}".format(WIKI_API_TOKEN)
}

In [5]:
test_id = '975900'

# Request parameters
params = {
    "action": "query",
    "format": "json",
    "prop": "pageprops",
    "pageids": test_id
}

# Request the pageprops for a page
response = requests.get(base_url, headers=headers, params=params).json()

print('Title: {} - WikidataID: {}'.format(response['query']['pages'][test_id]['title'], response['query']['pages'][test_id]['pageprops']['wikibase_item']))

Title: Ghosts of Mars - WikidataID: Q261700


#### Fetch Wikidata IDs for all movies
Create relation between the wiki_IDs from the CMU dataset to the WikiData IDs. We realized with a couple of test quesries that some wikipedia pages IDs are missing (indicated by a `missing` key), so we planned for that accordingly.

In [6]:
# Parameters
max_url_batch = 50

In [7]:
# We group all the IDs in batches of 50 to make the requests
batch_ids = ['|'.join(map(str, movie_df.index[i:i + max_url_batch])) for i in range(0, len(movie_df.index), max_url_batch)]
print("Batch requests: {}\nTotal movie count: {}".format(len(batch_ids), len(movie_df.index)))

Batch requests: 1635
Total movie count: 81741


In [16]:
iter = 0
wiki_ids = []
wdata_ids = []

# Sanity check: keep track of all wiki_ids that are missing a wikidata_id
missing_wdata_ids = []

for batch in batch_ids:

    params = {
        "action": "query",
        "format": "json",
        "prop": "pageprops",
        "pageids": batch
    }

    # Request the pageprops for all wiki_ids
    response = requests.get(base_url, headers=headers, params=params).json()

    # For each key in query.pages
    for key in response['query']['pages'].keys():
        wiki_ids.append(key)

        wdata_id = ''
        try:
            wdata_id = response['query']['pages'][key]['pageprops']['wikibase_item']
        except:
            missing_wdata_ids.append(key)
            pass
        wdata_ids.append(wdata_id)

    iter += 1
    print("Batch {} of {}".format(iter, len(batch_ids)))

    clear_output(wait=True)


Batch 1635 of 1635


In [17]:
# Sanity check for alignement
print(len(wiki_ids))
print(len(wdata_ids))

# Collect all the missing wiki_ids
missing_test = []
for i in range(len(wiki_ids)):
    if wdata_ids[i] == '':
        missing_test.append(wiki_ids[i])

# Check if all missing wiki_ids are in the missing_wdata_ids list
for i in range(len(missing_test)):
    if missing_test[i] not in missing_wdata_ids:
        print('Missing wiki_id not in missing_wdata_ids: {}'.format(missing_test[i]))

81741
81741


In [18]:
# Create table to store relations between wiki_ID and WikidataID
wpedia_wdata_df = pd.DataFrame(columns=['wiki_ID', 'wikidata_ID'])
wpedia_wdata_df['wiki_ID'] = wiki_ids
wpedia_wdata_df['wikidata_ID'] = wdata_ids

# Set the index to wiki_ID
wpedia_wdata_df.set_index('wiki_ID', inplace=True)

# Set all empty wikidata_ID to NaN
wpedia_wdata_df['wikidata_ID'].replace('', np.nan, inplace=True)

display(wpedia_wdata_df)

,wikidata_ID
wiki_ID,
18998739,NaN
9997961,NaN
20604092,NaN
31025505,NaN
77856,Q209170
...,...
31422084,Q8073901
32468537,Q965863
34474142,Q5505996


In [19]:
# Compute statistics
print("Total movies: {}".format(len(wpedia_wdata_df.index)))
print("Movies with WikidataID: {}".format(len(wpedia_wdata_df.dropna().index)))
print("Movies without WikidataID: {}".format(len(wpedia_wdata_df[wpedia_wdata_df['wikidata_ID'].isnull()].index)))
print("=> {}% of movies have a WikidataID".format(round(len(wpedia_wdata_df.dropna().index) / len(wpedia_wdata_df.index) * 100, 2)))

Total movies: 81741
Movies with WikidataID: 76573
Movies without WikidataID: 5168
=> 93.68% of movies have a WikidataID


In [20]:
# Save the table to a csv file
wpedia_wdata_df.to_csv(tmp_dir + 'wpedia_wdata.csv')

In [21]:
# Checking length of both dataframes
assert len(movie_df.index) == len(wpedia_wdata_df.index), "Dataframes have different lengths"

With 93.7% of the movies being linakble to a wikidata ID, we assume that some of the wikipedia pages were removed or updated since 2012.

This was our first attempt at the conversion of the wikipedia IDs to wikidata IDs and achieved decent results. However we will try to improve this conversion rate by going through the freebase IDs to get to the wikidata IDs.

### Part 2: Widikata to IMDB
We now fecth the IMDB ID (tconst) from the wikidata page's properties. We again use the REST API to do so instea of SPARQL queries.

In [22]:
# Import the wikipedia to wikidata table
wiki_to_wdata_df = pd.read_csv(tmp_dir + 'wpedia_wdata.csv', index_col='wiki_ID')
display(wiki_to_wdata_df)

,wikidata_ID
wiki_ID,
18998739,NaN
9997961,NaN
20604092,NaN
31025505,NaN
77856,Q209170
...,...
31422084,Q8073901
32468537,Q965863
34474142,Q5505996


In [23]:
# The IMDB ID is stored as property P345 in Wikidata
IMDB_claim = 'P345'

#### Test for a single ID

In [24]:
# Request setup
base_url = 'https://www.wikidata.org/w/api.php'

headers = {
    "Authorization": "Bearer {}".format(WIKI_API_TOKEN)
}

In [25]:
# Test fetching for a single wikidata imdb_id property
test_wikidata_id = 'Q261700'

# Request parameters
params = {
    "action": "wbgetentities",
    "ids": test_wikidata_id,
    "format": "json",
    "props": "claims"
}

# Request the pageprops for a page
response = requests.get(base_url, headers=headers, params=params).json()

print('WikidataID: {} - IMDB tconst: {}'.format(test_wikidata_id, response['entities'][test_wikidata_id]['claims'][IMDB_claim][0]['mainsnak']['datavalue']['value']))

WikidataID: Q261700 - IMDB tconst: tt0228333


#### Fetch for all wikidata IDs

In [26]:
# Parameters
max_url_batch = 50

In [27]:
# Dropping the NaN values
wiki_to_wdata_df_nonan = wiki_to_wdata_df.dropna().copy(deep=True)
display(wiki_to_wdata_df_nonan)

# For all wiki_id, we group all the wikidata IDs in batches of 50 to make the requests 
batch_wikidata_ids = ['|'.join(map(str, wiki_to_wdata_df_nonan.wikidata_ID[i:i + max_url_batch])) for i in range(0, len(wiki_to_wdata_df_nonan.values), max_url_batch)]
print("Batch requests: {}\nTotal movie count: {}".format(len(batch_wikidata_ids), len(wiki_to_wdata_df_nonan)))

,wikidata_ID
wiki_ID,
77856,Q209170
156558,Q607122
171005,Q114115
175024,Q729807
175026,Q1579725
...,...
31422084,Q8073901
32468537,Q965863
34474142,Q5505996


Batch requests: 1532
Total movie count: 76573


In [28]:
i = 0
wdata_ids = []
ttconsts = []

missing_ids_log = []

for batch in batch_wikidata_ids:
    print("Batch {} of {} (processed {} entries)".format(i + 1, len(batch_wikidata_ids), len(wdata_ids)))

    params = {
        "action": "wbgetentities",
        "ids": batch,
        "format": "json",
        "props": "claims"
    }

    # Request the pageprops for all wiki_ids
    response = requests.get(base_url, headers=headers, params=params).json()

    # For each key in entities
    for key in response['entities'].keys():
        wdata_ids.append(key)

        ttconst = ''
        try:
            ttconst = response['entities'][key]['claims'][IMDB_claim][0]['mainsnak']['datavalue']['value']
        except:
            missing_ids_log.append(key)
            pass
        ttconsts.append(ttconst)

    clear_output(wait=True)

    i += 1

Batch 1532 of 1532 (processed 76550 entries)


In [29]:
# Create table to store relations between wikidataID and ttconst
wdata_ttconst_df = pd.DataFrame(columns=['wikidata_ID', 'ttconst'])
wdata_ttconst_df['wikidata_ID'] = wdata_ids
wdata_ttconst_df['ttconst'] = ttconsts

# Set the index to wikidata_ID
wdata_ttconst_df.set_index('wikidata_ID', inplace=True)

# Set all empty ttconst to NaN
wdata_ttconst_df['ttconst'].replace('', np.nan, inplace=True)

display(wdata_ttconst_df)

,ttconst
wikidata_ID,
Q209170,tt0058331
Q607122,tt0255819
Q114115,tt0097499
Q729807,tt0020823
Q1579725,tt0021335
...,...
Q8073901,tt0120554
Q965863,tt0459759
Q5505996,tt0035905


In [30]:
# Save the table to a csv file
wdata_ttconst_df.to_csv(tmp_dir + 'wdata_ttconst.csv')

In [31]:
# Compute statistics
print("Total movies: {}".format(len(wdata_ttconst_df.index)))
print("Movies with IMDB IDs: {}".format(len(wdata_ttconst_df.dropna().index)))
print("Movies without IMDB IDs: {}".format(len(wdata_ttconst_df[wdata_ttconst_df['ttconst'].isnull()].index)))
print("=> {}% of movies have a IMDB ID linked".format(round(len(wdata_ttconst_df.dropna().index) / len(wdata_ttconst_df.index) * 100, 2)))

Total movies: 76573
Movies with IMDB IDs: 74855
Movies without IMDB IDs: 1718
=> 97.76% of movies have a IMDB ID linked


### Conversion Results
Linking the wikipedia IDs to the wikidata IDs and then to the IMDB IDs.

In [32]:
# Import two tables
wiki_to_wdata_df = pd.read_csv(tmp_dir + 'wpedia_wdata.csv', index_col='wiki_ID')
wdata_to_ttconst_df = pd.read_csv(tmp_dir + 'wdata_ttconst.csv', index_col='wikidata_ID')

# Left join the two tables on wikidata_ID
wiki_to_ttconst_df = wiki_to_wdata_df.join(wdata_to_ttconst_df, how='left', on='wikidata_ID')
display(wiki_to_ttconst_df)

# Compute the overall data loss
ttconst_loss = len(wiki_to_ttconst_df[wiki_to_ttconst_df['ttconst'].isnull()].index)
print("Total conversion success: {}% ({} out of {} movies lost)".format(round((1 - ttconst_loss / len(wiki_to_ttconst_df.index)) * 100, 2), ttconst_loss, len(wiki_to_ttconst_df.index)))

,wikidata_ID,ttconst
wiki_ID,,
18998739,NaN,NaN
9997961,NaN,NaN
20604092,NaN,NaN
31025505,NaN,NaN
77856,Q209170,tt0058331
...,...,...
31422084,Q8073901,tt0120554
32468537,Q965863,tt0459759
34474142,Q5505996,tt0035905


Total conversion success: 91.58% (6886 out of 81741 movies lost)


In [33]:
# Save the table to a csv file
filter_tmp = wiki_to_ttconst_df.reset_index()[['wiki_ID', 'ttconst']]
filter_tmp.columns = ['wikipedia_ID', 'IMDB_ID']
filter_tmp.set_index('wikipedia_ID', inplace=True)
filter_tmp.to_csv(tmp_dir + 'wpedia_ttconst_m1.csv')

## Method 2: Freebase conversion
Through [Wikibase queries](https://query.wikidata.org/), we extracted all freebase_ids data into a `.tsv` file. Our aim is to directly merge this file with the CMU dataset through the freebase ID and get the IMDB Id linked to a freebase entry. This file contains:
- `wikidata_id`: the wikidata ID of the entity
- `freebase_id`: the freebase ID of the entity
- `IMDB_ID`: the IMDB ID of the entity

We also have a file with the `wikidata_ids` and `TMDB_ID` of the movies, extracted in a similar way. This file contains:
- `wikidata_id`: the wikidata ID of the entity
- `TMDB_ID`: the TMDB ID of the entity

We will populate the first dataframe with the extra TMDB ID column from the second dataframe.

Additional filtering has been done to reduce the size of the file and only keep the entities that are relevent. More precisely, we kept:
- The freebase entities that have a IMDB ID attribute linked to them (i.e. the freebase entity has an IMDB ID)
- Only entries that followed the format of IMDB movie ids: `tt%` (there are IMDB related entries that had a different format, such as actors, characters, etc.)

In [50]:
# Import the freebase/imdb table
freebase_dir = raw_dir + 'wikidata/wikidata_freebase_imdb.tsv'

# Import the wikidata tmdb table
tmdb_dir = raw_dir + 'wikidata/wikidata_tmdb.tsv'

# Read the file into a DataFrame, add headers
freebase_ids = pd.read_csv(freebase_dir, sep='\t', names=['wikidata_ID', 'freebase_ID', 'IMDB_ID'], header=0)
tmdb_ids = pd.read_csv(tmdb_dir, sep='\t', names=['wikidata_ID', 'TMDB_ID'], header=0)
display(freebase_ids.head())
display(tmdb_ids.head())

,wikidata_ID,freebase_ID,IMDB_ID
0,http://www.wikidata.org/entity/Q595,/m/0hgqwpj,tt1675434
1,http://www.wikidata.org/entity/Q961,/m/027zfqr,tt0065035
2,http://www.wikidata.org/entity/Q990,/m/083whx,tt0120948
3,http://www.wikidata.org/entity/Q2345,/m/0m_tj,tt0050083
4,http://www.wikidata.org/entity/Q3997,/m/0jzsdw1,tt4168940


,wikidata_ID,TMDB_ID
0,http://www.wikidata.org/entity/Q109331,165
1,http://www.wikidata.org/entity/Q117378,156
2,http://www.wikidata.org/entity/Q131074,122
3,http://www.wikidata.org/entity/Q132863,12
4,http://www.wikidata.org/entity/Q134773,13


In [51]:
# Clean the wikidata_ID column by removing the first https://www.wikidata.org/wiki/ part
freebase_ids.wikidata_ID = freebase_ids.wikidata_ID.str[31:].copy(deep=True)
tmdb_ids.wikidata_ID = tmdb_ids.wikidata_ID.str[31:].copy(deep=True)

In [52]:
display(freebase_ids.sort_values(by="wikidata_ID"))
display(tmdb_ids.sort_values(by="wikidata_ID"))

,wikidata_ID,freebase_ID,IMDB_ID
40139,Q1000094,/m/0gkrmfs,tt0178997
80309,Q1000174,/m/0gjdn2_,tt0131068
80310,Q1000219,/m/08v34w,tt0241310
40140,Q1000394,/m/02rjv0t,tt0022478
53991,Q10005695,/m/012gbhgm,tt4443924
...,...,...,...
99603,Q999681,/m/02pm68d,tt0309396
116300,Q99977,/m/03cl7q4,tt7794218
40138,Q999770,/m/04j0wk9,tt0412639
2161,Q999778,/m/02621rq,tt0713624


,wikidata_ID,TMDB_ID
8718,Q1000094,5282
15819,Q1000174,372171
52511,Q1000210,91804
61208,Q1000394,199030
193286,Q100052082,615821
...,...,...
192988,Q99982342,429588
172526,Q9998284,365455
85177,Q99984183,746957
97702,Q999900,292750


In [53]:
# Check if there are any duplicates on freebase_ID
duplicates_on_f = freebase_ids.groupby('freebase_ID').apply(lambda x: len(x)).sort_values(ascending=False)
print("There are {} duplicates on freebase_ID".format(len(duplicates_on_f[duplicates_on_f > 1].index)))

# Do we have duplicates on both freebase_ID and IMDB_ID?
duplicates_on_f_i = freebase_ids.groupby(['freebase_ID', 'IMDB_ID']).apply(lambda x: len(x)).sort_values(ascending=False)
print("There are {} duplicates on freebase_ID and IMDB_ID".format(len(duplicates_on_f_i[duplicates_on_f_i > 1].index)))

There are 315 duplicates on freebase_ID
There are 13 duplicates on freebase_ID and IMDB_ID


We notice that we have duplicates in this file.

Out of curiosity, we check why does one freebase entry have multiple IMDB Ids. We manually check for one case and enter the IMDB IDs on the IMDB website, and we notice that these are different episodes of the same movie (eg: here, it is "Don't hug me I'm scared" which had 5 different episodes).

However, in our initial table, we have only a single entry for any movie: freebase_id is unique, so it should be linked to a single IMDB ID. We will therefore drop the duplicates.

Therefore, we will proceed the following way:
- We will keep an entry where we have the same `freebase_id` and `IMDB_ID`.
- We will drop entries where we have the same `freebase_id` but different `IMDB_ID` because this means they are unreliable.

In [54]:
# Is freebase id unique in movie_df?
print(f"In movie_df, freebase_ID is unique: {movie_df.freebase_ID.is_unique}")

# Show an example
display(freebase_ids[freebase_ids.freebase_ID == duplicates_on_f.index[0]])

In movie_df, freebase_ID is unique: True


,wikidata_ID,freebase_ID,IMDB_ID
35648,Q17011054,/m/0j8k82_,tt2501618
35649,Q17011054,/m/0j8k82_,tt3633758
36899,Q17011054,/m/0j8k82_,tt4287118
36900,Q17011054,/m/0j8k82_,tt5114582
36901,Q17011054,/m/0j8k82_,tt5114650


In [55]:
# We keep only one entry where we have the same `freebase_id` and `IMDB_ID`.
freebase_filter = freebase_ids.drop_duplicates(subset=['freebase_ID', 'IMDB_ID'], keep='first').copy(deep=True)

# We drop entries where we have the same `freebase_id` but different `IMDB_ID` because this means they are unreliable.
freebase_filter = freebase_filter.drop_duplicates(subset=['freebase_ID'], keep=False)
display(freebase_filter)

,wikidata_ID,freebase_ID,IMDB_ID
0,Q595,/m/0hgqwpj,tt1675434
1,Q961,/m/027zfqr,tt0065035
2,Q990,/m/083whx,tt0120948
3,Q2345,/m/0m_tj,tt0050083
4,Q3997,/m/0jzsdw1,tt4168940
...,...,...,...
157602,Q19571399,/m/012nmg7z,tt1411844
157603,Q1622572,/m/02qg338,tt0300877
157604,Q16255156,/m/0_qfb7q,tt10923898
157605,Q1077672,/m/027yzkv,tt2006485


In [61]:
# TMDB, we drop all duplicated wikidata_IDs because again, they are unreliable
tmdb_ids_cleared = tmdb_ids.drop_duplicates(subset=['wikidata_ID'], keep=False).copy(deep=True)
print(f"Size of tmdb_ids: {len(tmdb_ids.index)}, size of deuplicaets removal tmdb_ids_cleared: {len(tmdb_ids_cleared.index)}")

# Merge the two tables on wikidata_ID
freebase_tmdb_ids = freebase_filter.merge(tmdb_ids_cleared, how='left', on='wikidata_ID')
display(freebase_tmdb_ids)

# Note that we don't consider a TMDB_ID value if there is not IMDB_ID value linked 
# to a wikidata_ID, as without a rating an entry in the CMU is not useful for our purpose.
# This way we will never have a TMDB_ID without an IMDB_ID
print("Total missing TMDB IDs: {:.2%}".format(len(freebase_tmdb_ids[freebase_tmdb_ids['TMDB_ID'].isnull()].index) / len(freebase_tmdb_ids.index)))

Size of tmdb_ids: 230263, size of deuplicaets removal tmdb_ids_cleared: 229848


,wikidata_ID,freebase_ID,IMDB_ID,TMDB_ID
0,Q595,/m/0hgqwpj,tt1675434,77338
1,Q961,/m/027zfqr,tt0065035,NaN
2,Q990,/m/083whx,tt0120948,NaN
3,Q2345,/m/0m_tj,tt0050083,389
4,Q3997,/m/0jzsdw1,tt4168940,560182
...,...,...,...,...
156963,Q19571399,/m/012nmg7z,tt1411844,NaN
156964,Q1622572,/m/02qg338,tt0300877,NaN
156965,Q16255156,/m/0_qfb7q,tt10923898,NaN
156966,Q1077672,/m/027yzkv,tt2006485,NaN


Total missing TMDB IDs: 29.90%


In [63]:
movies_imdb = movie_df.reset_index().merge(freebase_tmdb_ids, on='freebase_ID', how='left')
display(movies_imdb)

,wikipedia_ID,freebase_ID,name,release_date,box_office,runtime,languages,countries,genres,wikidata_ID,IMDB_ID,TMDB_ID
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",Q261700,tt0228333,10016
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",Q16250726,tt0245916,784579
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",Q4978832,tt0094806,396302
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",Q7995657,tt0094320,33592
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",Q869644,tt0083949,11192
...,...,...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",Q6819873,tt1816585,117124
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",Q12125420,tt1606259,71771
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",Q4770308,tt0362411,285337
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",Q2663931,tt0113726,NaN


In [64]:
print(f'{movies_imdb.IMDB_ID.notna().mean()*100:.2f}% of movies in this database have corresponding Wikidata and IMDB pages.')

90.07% of movies in this database have corresponding Wikidata and IMDB pages.


In [65]:
# Save the filtered table to a csv file with, only the values that resulted in the merge
filt = movies_imdb[['wikipedia_ID', 'IMDB_ID', 'TMDB_ID']].copy(deep=True)
filt.to_csv(tmp_dir + 'wpedia_ttconst_m2.csv', index=False)

## More conversion rate? Assembling the two methods
What if we tried combining the two methods to improve our conversion rate?

In [109]:
# Import two tables
id_m1 = pd.read_csv(tmp_dir + 'wpedia_ttconst_m1.csv', index_col='wikipedia_ID')
id_m2 = pd.read_csv(tmp_dir + 'wpedia_ttconst_m2.csv', index_col='wikipedia_ID')

# Union of the two tables, remove wikipedia_IDs that were neither in m1 or m2
id_union = pd.concat([id_m1, id_m2]).dropna(subset=['IMDB_ID'])
display(id_union)

,IMDB_ID,TMDB_ID
wikipedia_ID,,
77856,tt0058331,NaN
156558,tt0255819,NaN
171005,tt0097499,NaN
175024,tt0020823,NaN
175026,tt0021335,NaN
...,...,...
35228177,tt1816585,117124.0
34980460,tt1606259,71771.0
9971909,tt0362411,285337.0


As a sanity check, we will check if the IMDB IDs we get from the two methods are the same. Namely, if there are duplicates after the union of the two tables that have different IMDB IDs, we will drop them because they are unreliable.

We curiously checked, we saw that for the WikiID entry `207863`, values `tt0324742` (from wikidata) and `tt0099317` (from freebase) and saw that it is the same movie title "Creature Comforts" but they have different release dates. To be careful and stick to our policy about uncertainty, we will drop this entry.

In [110]:
# Sanity check: see how many duplicates mismatch values
duplicate_check = id_union.drop(columns=['TMDB_ID']).groupby('wikipedia_ID').nunique()
mismatches = duplicate_check[duplicate_check > 1].dropna()
display(mismatches)
print(f"Total number of mismatching duplicates: {len(mismatches.index)}")

# We drop all of the mismatches entires from the union
id_union = id_union.drop(mismatches.index)
display(id_union)

,IMDB_ID
wikipedia_ID,
207863,2.0
858143,2.0
924115,2.0
1072373,2.0
1343188,2.0
...,...
35722781,2.0
35879747,2.0
36017798,2.0


Total number of mismatching duplicates: 142


,IMDB_ID,TMDB_ID
wikipedia_ID,,
77856,tt0058331,NaN
156558,tt0255819,NaN
171005,tt0097499,NaN
175024,tt0020823,NaN
175026,tt0021335,NaN
...,...,...
35228177,tt1816585,117124.0
34980460,tt1606259,71771.0
9971909,tt0362411,285337.0


In [112]:
# Drop duplicates of IMDB_ID where a not NaN TMDB_ID is prioritized

# Sort by IMDB_ID and then by TMDB_ID, nulls last
id_union_sorted = id_union.sort_values(by=['IMDB_ID', 'TMDB_ID'], ascending=[True, True], na_position='last').copy(deep=True)
display(id_union_sorted)

# Drop duplicates keeping the first (non-null TMDB_ID prioritized)
id_union_clean = id_union_sorted.drop_duplicates(subset=['IMDB_ID'], keep='first').copy(deep=True)
display(id_union_clean)

print(f'After dropping duplicates, we have {len(id_union_clean.index)} entries.')

,IMDB_ID,TMDB_ID
wikipedia_ID,,
22760465,ch0112831,NaN
185519,nm0158560,NaN
12754957,nm0704910,NaN
10111255,nm1016169,NaN
7669437,tt0000001,16612.0
...,...,...
17475562,tt9881364,NaN
8783319,tt9898504,NaN
8783319,tt9898504,NaN


,IMDB_ID,TMDB_ID
wikipedia_ID,,
22760465,ch0112831,NaN
185519,nm0158560,NaN
12754957,nm0704910,NaN
10111255,nm1016169,NaN
7669437,tt0000001,16612.0
...,...,...
22780907,tt9834564,834616.0
7316179,tt9855214,279530.0
17475562,tt9881364,1181096.0


After dropping duplicates, we have 76655 entries.


In [113]:
movies_imdb_final = movie_df.reset_index().merge(id_union_clean, on='wikipedia_ID', how='left')
display(movies_imdb_final)

print(f'{movies_imdb_final.IMDB_ID.notna().mean()*100:.2f}% of movies in this database have corresponding Wikidata and IMDB pages.')

,wikipedia_ID,freebase_ID,name,release_date,box_office,runtime,languages,countries,genres,IMDB_ID,TMDB_ID
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,10016.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",tt0245916,784579.0
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",tt0094806,396302.0
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",tt0094320,33592.0
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",tt0083949,11192.0
...,...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",tt1816585,117124.0
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",tt1606259,71771.0
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",tt0362411,285337.0
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",tt0113726,NaN


93.78% of movies in this database have corresponding Wikidata and IMDB pages.


## Results
We see that we managed to increase our conversion rate by combining these two methods by creating a union of the previous results. We have successfully create and will use this lookup table to convert 76654 movies from the CMU dataset to IMDB IDs for our project, with optional TMDB IDs that can be used later on.

In [115]:
# Save the results to a csv file
id_union_clean.reset_index().to_csv(processed_dir + 'movies_external_ids.csv', index=False)